<a href="https://colab.research.google.com/github/yhatpub/yhatpub/blob/dev/notebooks/fastai/lesson10_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fastai Lesson 10 on YHat.pub

This notebook picks up from [Fastai Fastbook 10 Text Generation](https://github.com/yhatpub/blogs/blob/main/10_nlp.ipynb) to [YHat.pub](https://yhat.pub)

To save your model, you'll need to export the pickled model after training, taking note of the export file size. Here is an example below
```
learn.export('/content/export.pkl')
!ls -l /content/export.pkl
from google.colab import files
files.download('/content/export.pkl')
```
Then upload it for public accessibility. Here is an example using [Google Drive](https://github.com/yhatpub/yhatpub#step-2-upload-your-model)

### Installs
The following cell installs pytorch, fastai and yhat_params, which is used to decorate your `predict` function.

In [2]:
# !pip install -q --upgrade --no-cache-dir torch
!pip install -q --upgrade --no-cache-dir fastai

# older version required to work with fastai
!pip install -q spacy==2.2.4
!pip install -q --no-cache-dir git+https://github.com/yhatpub/yhat_params.git@main

     |████████████████████████████████| 189 kB 5.4 MB/s 
     |████████████████████████████████| 56 kB 29.9 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 131 kB 5.3 MB/s 
     |████████████████████████████████| 62 kB 22.5 MB/s 
     |████████████████████████████████| 79 kB 51.1 MB/s 
     |████████████████████████████████| 8.1 MB 46.5 MB/s 
     |████████████████████████████████| 138 kB 55.3 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.26.0 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


Add the following since matplotlib needs to know where to write it's temp files

In [1]:
import os
import tempfile
os.environ["MPLCONFIGDIR"] = tempfile.gettempdir()

### Imports
**Warning** don't place `pip installs` and `imports` in the same cell. The imports might not work correctly if done that way.

In [3]:
from fastai.text.all import *
from yhat_params.yhat_tools import FieldType, inference_predict

### Download Model
Google drive does not allow direct downloads for files over 100MB, so you'll need to follow the snippet below to get the download url.

In [4]:
#cleanup from previous download
!rm -f uc*

#file copied from google drive
google_drive_url = "https://drive.google.com/file/d/10UWnRYV0H9h7T6_CJCashvfh31k9oiLK/view?usp=sharing"
import os
os.environ['GOOGLE_FILE_ID'] = google_drive_url.split('/')[5]
os.environ['GDRIVE_URL'] = f'https://docs.google.com/uc?export=download&id={os.environ["GOOGLE_FILE_ID"]}'
!echo "This is the Google drive download url $GDRIVE_URL"

This is the Google drive download url https://docs.google.com/uc?export=download&id=10UWnRYV0H9h7T6_CJCashvfh31k9oiLK


`wget` it from google drive. This script places the model in a `model` folder

In [5]:
!wget -q --no-check-certificate $GDRIVE_URL -r -A 'uc*' -e robots=off -nd
!mkdir -p models
!mv $(ls -S uc* | head -1) ./models/export.pkl

verify the model exists and double check the file size. **Warning** YHat is pretty finicky about where you place your models. Make sure you create a `model` directory and download your model(s) there  

In [6]:
!ls -l models

total 239228
-rw-r--r-- 1 root root 244964427 Nov  1 18:53 export.pkl


### Load your learner
The following is the equivalent of torch `torch.load` or ts `model.load_weights`

In [7]:
learn_inf = load_learner('models/export.pkl')
learn_inf.model.eval();

And write your predict function. Note, you will need to decorate your function with <a href="https://github.com/yhatpub/yhat_params">inference_predict</a> which takes 2 parameters, a `dic` for input and output.

**Info** These parameters are how YHat.pub maps your predict functions input/output of the web interface. The `dic` key is how you access the variable and the value is it's type. You can use autocomplete to see all the input/output types and more documentation on `inference_predict` is available at the link. 

In [8]:
input = {"text": FieldType.Text}
output = {"text": FieldType.Text}
N_WORDS = 20
N_SENTENCES = 1

@inference_predict(input=input, output=output)
def predict(params):
    preds = [learn_inf.predict(params["text"], N_WORDS, temperature=0.75) 
         for _ in range(N_SENTENCES)]
    output = "\n".join(preds)
    return {"text":output}

### Test
First, import `in_colab` since you only want to run this test in colab. YHat will use this colab in a callable API, so you don't want your test to run every time `predict` is called. Next, import `inference_test` which is a function to make sure your `predict` will run with YHat.

Now, inside a `in_colab` boolean, first get whatever test data you'll need, in this case, an image. Then you'll call your predict function, wrapped inside  `inference_test`, passing in the same params you defined above. If something is missing, you should see an informative error. Otherwise, you'll see something like
`Please take a look and verify the results`

In [9]:
from yhat_params.yhat_tools import in_colab, inference_test

if in_colab():
    inference_test(predict_func=predict, params={'text': "I liked this movie because"})

Wrote results to result.json duration: 3.425878 seconds
Please take a look and verify the results
{
    "text": "i liked this movie because it was filmed in Columbia , and i do n't think i have seen any Spielberg pictures in"
}


### That's it

If you run into errors, feel free to hop into Discord.

Otherwise, you'll now want to clear your outputs and save a public repo on Github